# Deploy

This notebook takes our trained models, deploys them as serving endpoints, and sets functions for a proper query.

## ML Model Deployment
We use MLflow's deploy client to simplify endpoint deployments

In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")
endpoint = client.create_endpoint(
    name="shm_3w_lightgbm",
    config={
        "served_entities": [
            {
                "name": "lightgbm", 
                "entity_name": "shm.3w.lightgbm",
                "entity_version": "2",
                "workload_size": "Small",
                "scale_to_zero_enabled": True
            }
        ],
        "traffic_config": {
            "routes": [
                {
                    "served_model_name": "lightgbm",
                    "traffic_percentage": 100
                }
            ]
        }
    }
)

# Prediction Tool

Let's declare a parameterized function that queries the model for a specific well and time range. We have also defined other tools using SQL to get information about our well.

In [0]:
spark.sql("""
    CREATE OR REPLACE FUNCTION shm.3w.predict_state(
        p_well_number INT DEFAULT 16 COMMENT "Well number between 1 and 50, must be integer", 
        p_num_minutes INT DEFAULT 1 COMMENT "Number of minutes to predict, must be under 30"
    )
    RETURNS TABLE (
        timestamp TIMESTAMP,
        prediction DOUBLE 
    )
    COMMENT "Generates hydrate predictions for the next couple of minutes"
    RETURN
    SELECT 
        f.timestamp,
        AI_QUERY(
            'hydrate_xgboost',
            request => NAMED_STRUCT(
            'T-JUS-CKP', f.`T-JUS-CKP`,
            'T-TPT', f.`T-TPT`,
            'P-TPT', f.`P-TPT`,
            'P-MON-CKP', f.`P-MON-CKP`,
            'P-PDG', f.`P-PDG`
            )
        ) AS prediction
    FROM shm.3w.filter_well_data(p_well_number, p_num_minutes)
"""
)